# Notebook for computing hashes, buckets and similarity values for the grid scheme. 

Utilizes the grid scheme

Incorporates:
* Hashing of trajectories using grid scheme
* Bucketing of hashes made from grid scheme
* Similarity computation between trajectories within buckets.
    * Both for DTW and Frechet

Produces:
* JSON file containing buckets
* Similarity values for trajectories within buckets


In [1]:

# Importing nescessary modules
import os
import sys

def find_project_root(target_folder="masteroppgave"):
    """Find the absolute path of a folder by searching upward."""
    currentdir = os.path.abspath("__file__")  # Get absolute script path
    while True:
        if os.path.basename(currentdir) == target_folder:
            return currentdir  # Found the target folder
        parentdir = os.path.dirname(currentdir)
        if parentdir == currentdir:  # Stop at filesystem root
            return None
        currentdir = parentdir  # Move one level up

# Example usage
project_root = find_project_root("masteroppgave")

if project_root:
    sys.path.append(project_root)
    print(f"Project root found: {project_root}")
else:
    raise RuntimeError("Could not find 'masteroppgave' directory")


#Other imports
from computation.grid_similarity import generate_grid_hash_similarity_with_bucketing
import json
import pandas as pd



Project root found: /Users/thomasnitsche/Developer/skole/master/masteroppgave
Project root found: /Users/thomasnitsche/Developer/skole/master/masteroppgave


# Rome


### DTW


In [2]:
city = "rome" #City to use
measure = "dtw" #Measure to use
res = 1.9 #Resolution to use
layers = 2 #Number of layers to use
size = 50 #Size of the grid to use


#Creates buckets and similarity matrix
similarities, bucket_system = generate_grid_hash_similarity_with_bucketing(
    city=city, res=res, layers=layers, measure=measure, size=size
)

### Write buckets to JSON and similarity matrix to CSV

In [4]:
# Write the bucket system to a JSON file
bucket_system_output_file_path = f"../../../results_hashed/buckets/grid/rome/rome_grid_resolution_{res}_layers_{layers}_trajectories_{size}.json"
with open(bucket_system_output_file_path, "w") as f:
    json.dump(bucket_system, f)
    
# Write similarity values to csv
output_path = "../../../results_hashed/similarity_values/grid/rome/dtw/"
similarities.to_csv(os.path.join(output_path, f"rome_grid_resolution_{res}_layers_{layers}_trajectories_{size}_{measure}.csv"))


### Frechet


In [ ]:
measure = "frechet"
similarities = generate_disk_hash_similarity(
    city="rome", diameter=1.6, layers=5, disks=100, measure=measure, size=500
)
output_path = f"similarity_values/disk/rome/disk_rome-{measure}.csv"
similarities.to_csv(os.path.abspath(output_path))

# Porto


In [ ]:
measure = "dtw"
similarities = generate_disk_hash_similarity(
    "porto", diameter=2.2, layers=4, disks=60, measure=measure, size=50
)
output_path = f"similarity_values/disk/porto/disk_porto-{measure}.csv"
similarities.to_csv(os.path.abspath(output_path))